<a href="https://colab.research.google.com/github/Satish1097/Colab_LLM_Learning/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q requests torch transformers sentencepiece accelerate

In [ ]:
!pip install -q -U bitsandbytes

In [3]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [4]:
hf_token = userdata.get('HUGGING_FACE_API_KEY')
login(hf_token, add_to_git_credential=True)

In [5]:
LLMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI3 = "microsoft/phi-3-mini-4k-instruct"
GEMMA2 = "google/gemma-2-2b-it"
QWEN2 = "Qwen/Qwen2-7B-Instruct"
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [6]:
messages=[
    {"role":'system',"content":"You're a helpful assistant"},
    {"role":'user',"content":"What is transformers"}
]

In [7]:
# Quantization config this allows us to load the model into memory and use less memory

quant_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(LLMA)
tokenizer.pad_token=tokenizer.eos_token
inputs=tokenizer.apply_chat_template(messages,return_tensors="pt").to("cuda")

In [ ]:
model=AutoModelForCausalLM.from_pretrained(LLMA,quantization_config=quant_config,device_map="auto")

In [ ]:
  memory=model.get_memory_footprint()/1e6
  print(f"Memory_footprint: {memory:,.1f}")

In [ ]:
model

In [ ]:
outputs=model.generate(inputs,max_new_tokens=80)
print(tokenizer.decode(outputs[0]))

In [25]:
# clean up

import torch
globals().pop('inputs', None)
globals().pop('model', None)
globals().pop('tokenizer', None)
torch.cuda.empty_cache()

torch.cuda.empty_cache()


In [26]:
# Wraping everything in a function and apply streaming

def generate(model,messages):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  tokenizer=AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token=tokenizer.eos_token
  inputs=tokenizer.apply_chat_template(messages,return_tensors="pt").to(device)
  streamer=TextStreamer(tokenizer)
  model=AutoModelForCausalLM.from_pretrained(LLMA,quantization_config=quant_config,device_map="auto")
  outputs=model.generate(inputs,max_new_tokens=80,streamer=streamer)
  del inputs,outputs,model,streamer,tokenizer
  torch.cuda.empty_cache



In [ ]:
generate(PHI3,messages)

In [ ]:
messages=[
    {"role":"user","content":"Hi, How are you"}
]

generate(GEMMA2, messages)
